In [1]:
import pandas as pd
import json
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import joblib
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
data = []
with open('hltb.jsonlines') as lines:
        for line in lines:
                data.append(json.loads(line))


In [3]:
#Convert nested dict into list of dicts format
#. I.e. {"Platform": {"Playstation 5": 5 hrs}} into [{"Platform_Playstation": 5}]
#test = df.iloc[0].Stats
def convert(list_dicts):
    total_dict = {}
    for h in list_dicts.keys():
        dict_1 = list_dicts[h].items()
        for i,j in dict_1:
            for k,l in j.items():
                total_dict.update({f'{h}_{i}_{k}': l})
    return  total_dict

#convert(test)

In [4]:
df = pd.DataFrame(data)
df = df.head(20000)
df['Genres'] = df['Genres'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
df.head()

,Name,Stats,steam_app_id,Release_date,Genres,Review_score
0,Borderlands 3,{'Additional Content': {'Moxxi's Heist of the ...,397540,2019-09-13,"First-Person, Action, Shooter",76
1,Dying Light,{'Additional Content': {'The Following': {'Pol...,239140,2015-01-27,"Action, Adventure, Open World, Survival Horror",78
2,Middle-Earth: Shadow of War,{'Additional Content': {'Blade of Galadriel': ...,356190,2017-10-10,"Action, Adventure",76
3,Counter-Strike: Global Offensive,"{'Multi-Player': {'Co-Op': {'Polled': '34', 'A...",730,2012-08-21,"First-Person, Shooter",74
4,Grand Theft Auto IV,{'Additional Content': {'The Lost and Damned':...,5152,2008-04-29,"Third-Person, Action, Sandbox, Shooter",82


In [5]:
total =  df.Stats.apply(lambda x: convert(x)).to_list()

#Let's keep only the primary keys for now as keeping all the keys would result in a very large and sparse dataset
keys_keep=  ['Single-Player_Main Story_Average']

total_keep= []
for g in total:
    temp = {}
    for i in keys_keep:
        for h in g:
            temp.update({i: g.get(i, '')})
    total_keep.append(temp)

In [10]:
def duration_to_minutes(duration_str):
  """Converts a duration string like '3h 18m' to minutes."""
  if isinstance(duration_str, str):
    parts = duration_str.split()
    minutes = 0
    for part in parts:
      if 'h' in part:
        minutes += int(part[:-1]) * 60  # Convert hours to minutes
      elif 'm' in part:
        minutes += int(part[:-1])  # Add minutes
    return minutes
  else:
    # Handle cases where duration is not a string (e.g., NaN)
    return 0  # or any other appropriate default value

In [11]:
df_format = pd.concat([df.drop(columns= ['Stats', 'steam_app_id']), pd.DataFrame(total_keep)], axis = 1)
df_final = df_format[['Name' , 'Genres', 'Single-Player_Main Story_Average']]
df_final = df_final.dropna()
#df_final['Genres'] = df_final['Genres'].apply(lambda x: x.split(',')[0])
label_encoder = LabelEncoder()
df_final['genre_encoded'] = label_encoder.fit_transform(df_final['Genres'])
df_final.info()
df_final.head(50)

<class 'pandas.core.frame.DataFrame'>
Index: 17063 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Name                              17063 non-null  object
 1   Genres                            17063 non-null  object
 2   Single-Player_Main Story_Average  17063 non-null  object
 3   genre_encoded                     17063 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 666.5+ KB


,Name,Genres,Single-Player_Main Story_Average,genre_encoded
0,Borderlands 3,"First-Person, Action, Shooter",23h 17m,456
1,Dying Light,"Action, Adventure, Open World, Survival Horror",18h 10m,38
2,Middle-Earth: Shadow of War,"Action, Adventure",20h 47m,17
3,Counter-Strike: Global Offensive,"First-Person, Shooter",,598
4,Grand Theft Auto IV,"Third-Person, Action, Sandbox, Shooter",27h 37m,1576
5,Cuphead,"Side, Action",10h 46m,1217
6,Marvel's Spider-Man: Game of the Year Edition,"Third-Person, Action, Adventure, Open World, S...",17h 27m,1524
7,Borderlands: The Pre-Sequel!,"First-Person, Action, Role-Playing, Shooter",18h 13m,452
8,Assassin's Creed Odyssey,"Third-Person, Action, Open World, Role-Playing",45h 10m,1562
9,Half-Life 2: Episode One,"First-Person, Shooter",3h 45m,598


In [12]:
#split dataset in features and target variable
feature_cols = ['genre_encoded', 'Single-Player_Main Story_Average']
X = df_final[feature_cols]
X['Single-Player_Main Story_Average'] = X['Single-Player_Main Story_Average'].apply(duration_to_minutes)
y = df_final["Name"]
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_418748\3715877909.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Single-Player_Main Story_Average'] = X['Single-Player_Main Story_Average'].apply(duration_to_minutes)


In [16]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X, y)
y_pred = knn.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
#Mostrar la matriz de confusión
#conf_matrix = confusion_matrix(y_test, y_pred)
#sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt='g')
#plt.title('Matriz de Confusión')
#plt.show()

# Evaluar la exactitud del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")

# Evaluo la precision del modelo
precision = precision_score(y_test, y_pred, average='weighted')
print(f"Precisión del modelo: {precision:.2f}")

# Evaluo el recall del modelo
recall = recall_score(y_test, y_pred, average='weighted')
print(f"Recall del modelo: {recall:.2f}")

# Evaluo el f1 score del modelo
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Score del modelo: {f1:.2f}")

Precisión del modelo: 0.79
Precisión del modelo: 0.77
Recall del modelo: 0.79
F1 Score del modelo: 0.78


c:\Users\USUARIO\FACULTAD\TPO CIENCIA DE DATOS\TPOCD\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USUARIO\FACULTAD\TPO CIENCIA DE DATOS\TPOCD\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
input_genero ='First-Person, Platform, Puzzle'  # Nuevo género
input_minutos = 200  # Nuevas minutos estimadass

# Verificar si el nuevo género está en las etiquetas del encoder
if input_genero in label_encoder.classes_:
    input_genero_encoded = label_encoder.transform([input_genero])[0]

    # Predecir el nombre del juego más cercano basado en género y horas
    prediction = knn.predict([[input_genero_encoded, input_minutos]])
    print(f"Juego recomendado: {prediction[0]}")
else:
    print(f"El género '{new_genre}' no está presente en los datos originales.")


Juego recomendado: Portal


c:\Users\USUARIO\FACULTAD\TPO CIENCIA DE DATOS\TPOCD\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [36]:
joblib.dump(knn, 'knn_model.pkl')

['knn_model.pkl']

In [37]:
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']